In [142]:
import glob
import json
from pprint import pprint

#### i had to rename some files becausde i didnt format the date properly D:

In [ ]:
filenames = glob.glob("./*.json")
file_dt = sorted([file[10:-5].split('-') for file in filenames], reverse=True)
print(file_dt)
for d,t in file_dt[:20]:
    date = d[:4] + '0' + d[-2] + '0' + d[-1] if len(d) < 9 else d
    if len(t) == 2:
        time = '0' + t[0] + '0' + t[1]
    elif len(t) == 3:
        time = '0' + t
    else:
        time = t
    orig = 'scraped-' + d + '-' + t + '.json'
    new = 'scraped-' + date + '-' + time + '.json'
    print(orig, new)
    # old_file = os.path.join(os.getcwd(), orig)
    # new_file = os.path.join(os.getcwd(), new)
    # os.rename(old_file, new_file)

#### for every date, time, lets see what the restaurants and deals are

In [113]:
def get_deal_info(deal_json):
    deals = set()
    for deal in deal_json:
        if deal["isSoldOut"] or not deal["isAvailable"]:
            continue
        else:
            deals.add(deal["title"])
    return deals

def get_deals(uber_json):
    deals = {res["name"]: get_deal_info(res["deals"])
            for res in uber_json}
    return deals

In [120]:
def deal_dif(prev_deals, deals):
    changes = {'added':[], 'removed': []}
    for resto in prev_deals:
        if resto not in deals:
            changes['removed'].append({resto: prev_deals[resto]})
            continue
        elif deals[resto] == prev_deals[resto]:
            continue
        else:
            changes['removed'].append({resto: prev_deals[resto].difference(deals[resto])})

    for resto in deals:
        if resto not in prev_deals:
            changes['added'].append({resto: deals[resto]})
            continue
        elif deals[resto] == prev_deals[resto]:
            continue
        else:
            changes['added'].append({resto: deals[resto].difference(prev_deals[resto])})
    return changes

In [ ]:
filenames = sorted(glob.glob("./*.json"))
prev_deals = []
daily_updates = {date: 0 for date in ['0401', '0402', '0403']}
for file in filenames:
    with open(file, 'r') as f:
        date, time = file[14:18], file[19:24]
        json_data = json.load(f)
        deals = get_deals(json_data)
        if not prev_deals:
            print(date, time, deals)
        else:
            daily_updates[date] += 1
            changes = deal_dif(prev_deals, deals)
            if changes['added'] or changes['removed']:
                print('Day', date, 'time', time)
                print(changes)
        prev_deals = deals
print(daily_updates)

In [152]:
def deals_in_range(mmdd, starttime, endtime):
    filenames = sorted(glob.glob(f"./scraped-2024{mmdd}-*.json"))
    for file in filenames:
        if starttime <= file[19:23] <= endtime:
            with open(file, 'r') as f:
                json_data = json.load(f)
                print('\n', file[19:23])
                pprint(get_deals(json_data))

deals_in_range('0404', '1200', '1500')


 1215
{"Anne's Artisan Coffee (22 4th Ave)": {'Buttered Bagel'},
 'Brekkie Bagels (108 Montague St)': {'Bacon Egg and Cheese Bagel'},
 'Pio Bagel - Flatbush': {'Butter Bagel Sandwich',
                          'Cream Cheese Sandwich',
                          'Two Eggs, Bacon, and Cheese Sandwich'},
 'Pure Green Boerum Place (Brooklyn)': {'Coconut Chia Acai Bowl',
                                        'Pink Dragon Smoothie',
                                        'Pure Pina Colada Smoothie',
                                        'Pure Pitaya Bowl',
                                        'Tropical Breeze Smoothie'},
 "Seth's Sandwiches (22 4th Ave)": {'Onion Rings'},
 "Sunny &amp; Fine's Breakfast Burritos": {'Bacon Breakfast Burrito'},
 'Tio Pio': {'Chocolate Cake'}}

 1241
{"Anne's Artisan Coffee (22 4th Ave)": {'Buttered Bagel'},
 'Brekkie Bagels (108 Montague St)': {'Bacon Egg and Cheese Bagel'},
 'Juice Press (124 Montague Street )': {'Advanced Berry Blend',
              